In [1]:
import os
import re

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

In [8]:
valid_CMA = [
    # "St. John's", 
    "Halifax", 
    "Québec", 
    "Montréal", 
    "Ottawa - Gatineau (Ontario part / partie de l'Ontario)", 
    "Toronto", 
    # "Kitchener - Cambridge - Waterloo", 
    # "Hamilton", 
    # "London", 
    "Winnipeg", 
    # "Regina", 
    # "Saskatoon", 
    "Edmonton", 
    "Calgary", 
    "Vancouver", 
    # "Victoria"
]

valid_mont = [
    "Montréal", 
    "Montréal-Est", 
    "Montréal-Ouest", 
    "Westmount", 
    "Côte-Saint-Luc", 
    "Mont-Royal", 
    "Hampstead", 
    "Dorval", 
    "Pointe-Claire", 
    "Dollard-Des Ormeaux", 
    "Kirkland", 
    "Beaconsfield", 
    "Baie-D'Urfé", 
    "Sainte-Anne-de-Bellevue", 
    "Senneville"
]

RENAME_CSD_MAP = {
    "Québec": "Quebec City",
}

RENAME_REGION_MAP = {
    "Montréal": "Greater Montreal",
    "Québec": "Quebec City",
    "Ottawa - Gatineau (Ontario part / partie de l'Ontario)": "Greater Ottawa",
    "Vancouver": "Metro Vancouver",
}

Filter for the CMAs that we are interested in, and replace the Toronto CMA with the Greater Toronto Area region

In [6]:
gdf_cma = gpd.read_file('../data/boundaries/cen_cma/')

In [7]:
gdf_regions = gdf_cma.loc[gdf_cma['CMANAME'].isin(valid_CMA)].reset_index(drop=True)

# Substitute Toronto CMA for GTA
gdf_gta = gpd.read_file('../data/boundaries/greater-toronto-area.gpkg')
gdf_gta = gdf_gta.to_crs(gdf_regions.crs)
gdf_gta['CMANAME'] = 'Greater Toronto Area'
gdf_regions = pd.concat([gdf_regions, gdf_gta], ignore_index=True)
gdf_regions = gdf_regions[gdf_regions['CMANAME'] != 'Toronto']

gdf_regions = gdf_regions.rename(columns={'CMANAME': 'REGION_NAME'})
gdf_regions = gdf_regions.drop(columns=['DGUID', 'DGUIDP', 'CMATYPE', 'LANDAREA', 'PRUID'])
gdf_regions.to_file('../data/boundaries/valid_regions.gpkg', driver='GPKG')

Filter for the CSDs which are contained within a CMA/region, then replace all the CSDs on the Island of Montreal with the combined geography

In [9]:
gdf_csd = gpd.read_file('../data/boundaries/cen_csd/')
gdf_regions = gpd.read_file('../data/boundaries/valid_regions.gpkg')

In [10]:
# Spatial join to get intersecting pairs
gdf_joined = gpd.sjoin(gdf_csd, gdf_regions, how='inner', predicate='intersects')

# Compute area of each CSD (before intersection)
gdf_csd_area = gdf_csd[['CSDUID', 'geometry']].copy()
gdf_csd_area['area_csd'] = gdf_csd_area.geometry.area

# Merge area info into joined set
gdf_joined = gdf_joined.merge(gdf_csd_area[['CSDUID', 'area_csd']], on='CSDUID')

# Compute intersection geometry between each CSD and region
gdf_joined['intersection_geom'] = gdf_joined.geometry.intersection(gdf_regions.unary_union)
gdf_joined['area_intersection'] = gdf_joined['intersection_geom'].area

# Filter to those with ≥10% overlap
gdf_filtered = gdf_joined[gdf_joined['area_intersection'] / gdf_joined['area_csd'] >= 0.10]

# Select final columns and write to file
gdf_filtered = gdf_filtered[['CSDUID', 'CSDNAME', 'geometry']]
gdf_filtered.to_file('../data/boundaries/valid_csds.gpkg', driver='GPKG')

In [11]:
# Load valid CSDs
gdf_csd = gpd.read_file('../data/boundaries/valid_csds.gpkg')

# 1. Filter CSDs that are part of the Island of Montreal
montreal_csds = gdf_csd[gdf_csd['CSDNAME'].isin(valid_mont)]

# 2. Remove them from the full set
gdf_csd_cleaned = gdf_csd[~gdf_csd['CSDNAME'].isin(valid_mont)]

# 3. Dissolve Montreal CSDs into a single unified geometry
montreal_geom = montreal_csds.unary_union
montreal_unified = gpd.GeoDataFrame({
    'CSDNAME': ['Island of Montreal'],
    'geometry': [montreal_geom]
}, crs=gdf_csd.crs)

# 4. Add missing columns (set to None) and preserve schema
for col in gdf_csd_cleaned.columns:
    if col not in montreal_unified.columns:
        montreal_unified[col] = None
montreal_unified = montreal_unified[gdf_csd_cleaned.columns]  # column order match

# 5. Merge and save
final_gdf = pd.concat([gdf_csd_cleaned, montreal_unified], ignore_index=True)
final_gdf.to_file('../data/boundaries/valid_csds.gpkg', driver='GPKG')


Clean up the ridings so that we only get what intersects with the shoreline boundaries as determined by the province and territories file.

In [3]:
gdf_ridings = gpd.read_file('../data/fed_ridings.gpkg')
gdf_canada = gpd.read_file('../data/boundaries/canada_simplified_fixed.gpkg')

In [4]:
# Reproject both to the same CRS
target_crs = "EPSG:3347"  # Statistics Canada Lambert
gdf_ridings = gdf_ridings.to_crs(target_crs)
gdf_canada = gdf_canada.to_crs(target_crs)

# Ensure gdf_canada is a single polygon (in case it's multi-feature)
canada_union = gdf_canada.unary_union
gdf_canada = gpd.GeoDataFrame(geometry=[canada_union], crs=target_crs)

# Use GeoPandas overlay with intersection — this is fast & preserves attributes
gdf_clipped = gpd.overlay(gdf_ridings, gdf_canada, how='intersection')

# Write to file
gdf_clipped.to_file('../data/fed_ridings_clipped.gpkg', driver='GPKG')

/tmp/ipykernel_602683/2053149256.py:11: UserWarning: `keep_geom_type=True` in overlay resulted in 69505 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  gdf_clipped = gpd.overlay(gdf_ridings, gdf_canada, how='intersection')


Tag every riding with a CSD and region provided there is at least 50% overlapping area

In [12]:
gdf_regions = gpd.read_file('../data/boundaries/valid_regions.gpkg')
gdf_csd = gpd.read_file('../data/boundaries/valid_csds.gpkg')
gdf_ridings = gpd.read_file('../data/fed_ridings_clipped.gpkg')

In [14]:
# Ensure all geometries are valid and in same CRS
target_crs = "EPSG:3347"  # NAD83 Statistics Canada Lambert

gdf_regions = gdf_regions.to_crs(target_crs)
gdf_csd = gdf_csd.to_crs(target_crs)
gdf_ridings = gdf_ridings.to_crs(target_crs)

# Calculate riding areas (now only land area)
gdf_ridings['riding_area'] = gdf_ridings.geometry.area

def get_best_overlap(target_gdf, source_gdf, source_name_col, min_coverage=0.5):
    """
    Find best overlapping feature for each target geometry.
    Returns a Series with the best matches.
    """
    results = pd.Series(index=target_gdf.index, dtype='object')
    
    for idx, target_geom in target_gdf.geometry.items():
        # Find all intersecting features
        intersections = source_gdf[source_gdf.intersects(target_geom)].copy()
        
        if len(intersections) == 0:
            continue
            
        # Calculate intersection areas
        intersections['intersection_area'] = intersections.geometry.intersection(target_geom).area
        intersections['coverage'] = intersections['intersection_area'] / target_gdf.loc[idx, 'riding_area']
        
        # Get best match
        best_match = intersections.loc[intersections['coverage'].idxmax()]
        
        if best_match['coverage'] >= min_coverage:
            results.at[idx] = best_match[source_name_col]
            
    return results

# Add CSD column (using clipped ridings)
gdf_ridings['CSD'] = get_best_overlap(gdf_ridings, gdf_csd, 'CSDNAME')

# Add region column (using clipped ridings)
gdf_ridings['region'] = get_best_overlap(gdf_ridings, gdf_regions, 'REGION_NAME')

# Drop temporary column
gdf_ridings = gdf_ridings.drop(columns=['riding_area'])

# Rename certain tags
gdf_ridings['CSD'] = gdf_ridings['CSD'].apply(lambda x: RENAME_CSD_MAP[x] if x in RENAME_CSD_MAP else x)
gdf_ridings['region'] = gdf_ridings['region'].apply(lambda x: RENAME_REGION_MAP[x] if x in RENAME_REGION_MAP else x)

# Save results
gdf_ridings.to_file('../data/fed_ridings_tagged.gpkg', layer='fed_ridings_tagged', driver='GPKG', index=False)
gdf_ridings.drop(columns='geometry').to_csv('../data/fed_ridings_tagged.csv', index=False)